# Computer Vision Part 1 Face detection 

## Viola-Jones Algorithm:
`Training` and `Detection`<br>
### Detection:
`Grey Scale` conversion and Feature detection, when feature is found the feature is transfered to original image.<br> 
<b>fig. Viola-Jones Algorithm after n iterations to find the features</b>
![Screenshot%20from%202018-08-24%2011-07-30.png](Screenshot%20from%202018-08-24%2011-07-30.png)

![Screenshot%20from%202018-08-24%2011-09-52.png](Screenshot%20from%202018-08-24%2011-09-52.png)

#### Haar-like Features:
derived from `haar wavelets`<br>
The features are `edge features,line features and four rectangle features`<br>
<b>fig. haar like features</b>
![Screenshot%20from%202018-08-24%2011-13-46.png](Screenshot%20from%202018-08-24%2011-13-46.png)

![Screenshot%20from%202018-08-24%2011-17-52.png](Screenshot%20from%202018-08-24%2011-17-52.png)

The Viola-Jones Algorithm uses haar like features to detect the features.<br>
For example lets consider the nose of the person in the figure. <br> the pixel matrix and look for the `edge feature`, comparing with the edge feature from haar like features we see that 0.0 is white and 1.0 is black, now in picture(grey scale) we get the points and say we calculate the average of the `light` part and `dark` part and then calculate the difference between, if the difference is surpass the `threshold` given in the algorithm `0.3` then we can consider that we detected the haar like edge.<br>
<b>Fig. Haar like edge detection example</b>
![Screenshot%20from%202018-08-24%2011-20-38.png](Screenshot%20from%202018-08-24%2011-20-38.png)

#### Integral Image:
The above step to detect the haar like features can be computationally very big.<br>
To make it less complex we use `Integral Image`.<br>
Every pixel in the integral image is the sum of the pixels that are in the rectangle that this pixel makes. This makes us very easy to find the sums to detect the haar like features than what it would be in the raw image.<br>
<b>Fig. Using Integral images to decrease the complexity of haar like feature detection.</b>
![Screenshot%20from%202018-08-24%2011-40-41.png](Screenshot%20from%202018-08-24%2011-40-41.png)


### Training:
Train on the features to determine the features and set the threshold to detect something as a feature.<br>
We scale the image down to `24*24` pixels and then try to detect the features using haar like features. or we do it other way round, i.e we scale the features up to the size of image.<br>
Thus we train on lots of faces. And also on pictures without any faces as then it will be able to train itself and decrease the false positives.<br>
<b>Figures</b>
![Screenshot%20from%202018-08-24%2015-31-00.png](Screenshot%20from%202018-08-24%2015-31-00.png)
![Screenshot%20from%202018-08-24%2015-32-26.png](Screenshot%20from%202018-08-24%2015-32-26.png)
![Screenshot%20from%202018-08-24%2015-25-42.png](Screenshot%20from%202018-08-24%2015-25-42.png)


### Adaptive Boosting(Adaboost):
Even in `24*24` pixels picture the number of features is over 180,000 so on huge dataset the training will become computationally very hard.<br>
`Adaboost`:<br>
We take our image and put it in the classifier, each of the features are individually `weak classifiers` with the `ensemble method` we determine the strong classifier.<br>


In order to find out the most important features, we use `adaboost`. say one feature detected faces in following way: with false positives and false negatives, true positives and true negatives:<br>



Adaboost---> will help it decrease the false negatives and false positives(some other feature will work on the area that this feature was weak on to improve the performance. i.e the other feature will be the one that best compliments the selected feature.)<br>
Hence in next round of training it will give more weights to where the errors were made.ie. the importance of the the errors for next haar like feature detection becomes more.<br>
<b>Figures</b>
![Screenshot%20from%202018-08-24%2015-53-12.png](Screenshot%20from%202018-08-24%2015-53-12.png)
![Screenshot%20from%202018-08-24%2015-54-12.png](Screenshot%20from%202018-08-24%2015-54-12.png)


### Cascading:
We select a subwindow and then try one of the most important features on it, if the feature is not present, we discard the sub window.<br>
However if there is the feature present in the the subwindow we check it for the next important feature.<br>
This way we proceed...<br>
Infact while looking for the features, it looks for the top 5(say) features at a time in reality as dropping a subwindow just cuz we were not able to detect one feature can be risky.<br>
![Screenshot%20from%202018-08-24%2016-00-54.png](Screenshot%20from%202018-08-24%2016-00-54.png)

In [3]:
#library import 
import cv2

#Loading Cascades
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
smile_cascade = cv2.CascadeClassifier('haarcascade_smile.xml')

#Detecting features
def detect(gray,frame):
    face = face_cascade.detectMultiScale(gray,1.3,5)
    for (x,y,w,h) in face:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h,x:x+w]
        roi_color = frame[y:y+h,x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray,1.1,22)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
        smile = smile_cascade.detectMultiScale(roi_gray,1.7,22)
        for (sx,sy,sw,sh) in smile:
            cv2.rectangle(roi_color,(sx,sy),(sx+sw,sy+sh),(0,0,255),2)
    return frame

#deploying webcam
video_capture = cv2.VideoCapture(0)
while True:
    _,frame = video_capture.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    canvas = detect(gray,frame)
    cv2.imshow('Video',canvas)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()



![Screenshot%20%2849%29.png](Screenshot%20%2849%29.png)
![Screenshot%20%2852%29.png](Screenshot%20%2852%29.png)
